In [1]:
import pandas as pd

In [2]:
df_training = pd.read_csv('./cleaned/clean_private_dataset.csv')

In [3]:
df_park=pd.read_csv('./外部資料/公園.csv')
df_dep=pd.read_csv('./外部資料/百貨公司.csv')
df_tmarket=pd.read_csv('./外部資料/菜市場.csv')
df_bad=pd.read_csv('./外部資料/嫌惡設施基本資料.csv')

df_bus=pd.read_csv('./外部資料/公車站點資料.csv')
df_bike=pd.read_csv('./外部資料/腳踏車站點資料.csv')
df_train=pd.read_csv('./外部資料/火車站點資料.csv')
df_mrt=pd.read_csv('./外部資料/捷運站點資料_new.csv')

park

In [4]:
from geopy.distance import geodesic

def distance(x):
    global ori     # global vars, 起點, i.e each row in df
    lat1, lon1 = ori
    lat2, lon2 = x['lat'], x['lon']

    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)

    d = geodesic(coords_1, coords_2).meters  # 使用更快速的距離計算方式
    return d


In [5]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_park 是包含公園經緯度的 DataFrame
bus_coords = df_park[['Latitude', 'Longitude']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米


/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between atm and house...')
ori = (0, 0)
bus_stn = df_park[['Latitude', 'Longitude']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all atm station

with ThreadPoolExecutor() as executor:
    df_training['d_park'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between atm and house...


In [7]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_atm 欄位轉換成浮點數格式
df_training['d_park'] = df_training['d_park'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,land_area,building_area,main_building_area,balcony_area,auxiliary_area,floor,total_floor,parking_area,parking_number,d_park
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,-0.54,0.04,0.13,0.04,-0.44,12,14,-0.82,0.00,1.55
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.75,-0.20,-0.46,-0.16,-0.44,4,7,0.31,1.00,5.76
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.48,0.21,0.83,2.46,-0.44,6,7,-0.82,0.00,0.09
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.39,-0.69,0.05,-1.64,-0.44,4,4,-0.82,0.00,3.42
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.71,-0.90,-0.97,-0.33,-0.02,3,8,-0.82,0.00,2.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,-0.16,0.17,-0.50,0.95,0.87,25,25,1.66,2.00,4.38
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,1.12,2.49,2.72,1.11,-0.13,6,17,4.85,3.00,2.20
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,-0.05,-0.17,0.24,-0.08,-0.44,14,14,-0.82,0.00,7.97
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,-0.09,0.45,-0.26,2.46,-0.44,2,15,-0.82,0.00,5.29


In [8]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['park_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_park'] < 0.5, 'park_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,building_area,main_building_area,balcony_area,auxiliary_area,floor,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.04,0.13,0.04,-0.44,12,14,-0.82,0.00,1.55,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,-0.20,-0.46,-0.16,-0.44,4,7,0.31,1.00,5.76,False
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.21,0.83,2.46,-0.44,6,7,-0.82,0.00,0.09,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,-0.69,0.05,-1.64,-0.44,4,4,-0.82,0.00,3.42,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.90,-0.97,-0.33,-0.02,3,8,-0.82,0.00,2.42,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.17,-0.50,0.95,0.87,25,25,1.66,2.00,4.38,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,2.49,2.72,1.11,-0.13,6,17,4.85,3.00,2.20,False
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,-0.17,0.24,-0.08,-0.44,14,14,-0.82,0.00,7.97,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,0.45,-0.26,2.46,-0.44,2,15,-0.82,0.00,5.29,False


In [9]:
unique_values = df_training['park_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


dep

In [10]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_dep[['Latitude', 'Longitude']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [11]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between atm and house...')
ori = (0, 0)
bus_stn = df_dep[['Latitude', 'Longitude']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all atm station

with ThreadPoolExecutor() as executor:
    df_training['d_dep'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between atm and house...


In [12]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_atm 欄位轉換成浮點數格式
df_training['d_dep'] = df_training['d_dep'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,main_building_area,balcony_area,auxiliary_area,floor,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.13,0.04,-0.44,12,14,-0.82,0.00,1.55,False,2.77
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,-0.46,-0.16,-0.44,4,7,0.31,1.00,5.76,False,4.55
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.83,2.46,-0.44,6,7,-0.82,0.00,0.09,True,1.72
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.05,-1.64,-0.44,4,4,-0.82,0.00,3.42,False,4.94
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.97,-0.33,-0.02,3,8,-0.82,0.00,2.42,False,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,-0.50,0.95,0.87,25,25,1.66,2.00,4.38,False,5.51
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,2.72,1.11,-0.13,6,17,4.85,3.00,2.20,False,1.67
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.24,-0.08,-0.44,14,14,-0.82,0.00,7.97,False,0.72
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,-0.26,2.46,-0.44,2,15,-0.82,0.00,5.29,False,7.44


In [13]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['dep_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_park'] < 0.5, 'dep_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,balcony_area,auxiliary_area,floor,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.04,-0.44,12,14,-0.82,0.00,1.55,False,2.77,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,-0.16,-0.44,4,7,0.31,1.00,5.76,False,4.55,False
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,2.46,-0.44,6,7,-0.82,0.00,0.09,True,1.72,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,-1.64,-0.44,4,4,-0.82,0.00,3.42,False,4.94,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.33,-0.02,3,8,-0.82,0.00,2.42,False,4.11,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.95,0.87,25,25,1.66,2.00,4.38,False,5.51,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,1.11,-0.13,6,17,4.85,3.00,2.20,False,1.67,False
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,-0.08,-0.44,14,14,-0.82,0.00,7.97,False,0.72,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,2.46,-0.44,2,15,-0.82,0.00,5.29,False,7.44,False


In [14]:
unique_values = df_training['dep_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


tmarket

In [15]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_tmarket[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [16]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between atm and house...')
ori = (0, 0)
bus_stn = df_tmarket[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all atm station

with ThreadPoolExecutor() as executor:
    df_training['d_tmarket'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between atm and house...


In [17]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_atm 欄位轉換成浮點數格式
df_training['d_tmarket'] = df_training['d_tmarket'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,auxiliary_area,floor,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,-0.44,12,14,-0.82,0.00,1.55,False,2.77,False,0.81
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,-0.44,4,7,0.31,1.00,5.76,False,4.55,False,0.22
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,-0.44,6,7,-0.82,0.00,0.09,True,1.72,True,0.20
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,-0.44,4,4,-0.82,0.00,3.42,False,4.94,False,0.46
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.02,3,8,-0.82,0.00,2.42,False,4.11,False,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.87,25,25,1.66,2.00,4.38,False,5.51,False,0.58
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,-0.13,6,17,4.85,3.00,2.20,False,1.67,False,0.25
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,-0.44,14,14,-0.82,0.00,7.97,False,0.72,False,1.99
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,-0.44,2,15,-0.82,0.00,5.29,False,7.44,False,1.24


In [18]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['tmarket_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_tmarket'] < 0.5, 'tmarket_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,floor,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,12,14,-0.82,0.00,1.55,False,2.77,False,0.81,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,4,7,0.31,1.00,5.76,False,4.55,False,0.22,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,6,7,-0.82,0.00,0.09,True,1.72,True,0.20,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,4,4,-0.82,0.00,3.42,False,4.94,False,0.46,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,3,8,-0.82,0.00,2.42,False,4.11,False,0.46,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,25,25,1.66,2.00,4.38,False,5.51,False,0.58,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,6,17,4.85,3.00,2.20,False,1.67,False,0.25,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,14,14,-0.82,0.00,7.97,False,0.72,False,1.99,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,2,15,-0.82,0.00,5.29,False,7.44,False,1.24,False


In [19]:
unique_values = df_training['tmarket_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


bad

In [20]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_bad[['Latitude', 'Longitude']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [21]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between atm and house...')
ori = (0, 0)
bus_stn = df_dep[['Latitude', 'Longitude']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all atm station

with ThreadPoolExecutor() as executor:
    df_training['d_bad'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between atm and house...


In [22]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_atm 欄位轉換成浮點數格式
df_training['d_bad'] = df_training['d_bad'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,total_floor,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,14,-0.82,0.00,1.55,False,2.77,False,0.81,False,2.65
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,7,0.31,1.00,5.76,False,4.55,False,0.22,True,3.23
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,7,-0.82,0.00,0.09,True,1.72,True,0.20,True,0.62
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,4,-0.82,0.00,3.42,False,4.94,False,0.46,True,1.91
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,8,-0.82,0.00,2.42,False,4.11,False,0.46,True,2.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,25,1.66,2.00,4.38,False,5.51,False,0.58,False,2.82
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,17,4.85,3.00,2.20,False,1.67,False,0.25,True,1.75
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,14,-0.82,0.00,7.97,False,0.72,False,1.99,False,1.21
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,15,-0.82,0.00,5.29,False,7.44,False,1.24,False,2.79


In [23]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位bad設為False
df_training['bad_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_bad'] < 0.5, 'bad_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,parking_area,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,-0.82,0.00,1.55,False,2.77,False,0.81,False,2.65,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.31,1.00,5.76,False,4.55,False,0.22,True,3.23,False
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,-0.82,0.00,0.09,True,1.72,True,0.20,True,0.62,False
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,-0.82,0.00,3.42,False,4.94,False,0.46,True,1.91,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,-0.82,0.00,2.42,False,4.11,False,0.46,True,2.09,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,1.66,2.00,4.38,False,5.51,False,0.58,False,2.82,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,4.85,3.00,2.20,False,1.67,False,0.25,True,1.75,False
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,-0.82,0.00,7.97,False,0.72,False,1.99,False,1.21,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,-0.82,0.00,5.29,False,7.44,False,1.24,False,2.79,False


In [24]:
unique_values = df_training['bad_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


bus

In [25]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_bus[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [26]:
from geopy.distance import geodesic

def distance(x):
    global ori     # global vars, 起點, i.e each row in df
    lat1, lon1 = ori
    lat2, lon2 = x['lat'], x['lon']

    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)

    d = geodesic(coords_1, coords_2).meters  # 使用更快速的距離計算方式
    return d


In [27]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_bus 是包含公車站經緯度的 DataFrame
bus_coords = df_bus[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371


In [28]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between bus and house...')
ori = (0, 0)
bus_stn = df_bus[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_bus'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between bus and house...


In [29]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_bike 欄位轉換成浮點數格式
df_training['d_bus'] = df_training['d_bus'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,parking_number,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.00,1.55,False,2.77,False,0.81,False,2.65,False,0.23
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,1.00,5.76,False,4.55,False,0.22,True,3.23,False,0.09
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.00,0.09,True,1.72,True,0.20,True,0.62,False,0.09
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.00,3.42,False,4.94,False,0.46,True,1.91,False,0.01
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.00,2.42,False,4.11,False,0.46,True,2.09,False,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,2.00,4.38,False,5.51,False,0.58,False,2.82,False,0.19
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,3.00,2.20,False,1.67,False,0.25,True,1.75,False,0.08
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.00,7.97,False,0.72,False,1.99,False,1.21,False,0.04
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,0.00,5.29,False,7.44,False,1.24,False,2.79,False,0.18


In [30]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['bus_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_bus'] < 0.5, 'bus_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_park,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,1.55,False,2.77,False,0.81,False,2.65,False,0.23,True
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,5.76,False,4.55,False,0.22,True,3.23,False,0.09,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.09,True,1.72,True,0.20,True,0.62,False,0.09,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,3.42,False,4.94,False,0.46,True,1.91,False,0.01,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,2.42,False,4.11,False,0.46,True,2.09,False,0.10,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,4.38,False,5.51,False,0.58,False,2.82,False,0.19,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,2.20,False,1.67,False,0.25,True,1.75,False,0.08,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,7.97,False,0.72,False,1.99,False,1.21,False,0.04,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,5.29,False,7.44,False,1.24,False,2.79,False,0.18,True


In [31]:
#確認
unique_values = df_training['bus_distance_less_than_500m'].unique()
print(unique_values)

[ True False]


bike

In [32]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_bike[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [33]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_bus 是包含公車站經緯度的 DataFrame
bus_coords = df_bike[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371


In [34]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between bike and house...')
ori = (0, 0)
bus_stn = df_bus[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_bike'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between bike and house...


In [35]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_bike 欄位轉換成浮點數格式
df_training['d_bike'] = df_training['d_bike'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,park_distance_less_than_500m,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,2.77,False,0.81,False,2.65,False,0.23,True,0.16
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,False,4.55,False,0.22,True,3.23,False,0.09,True,0.15
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,1.72,True,0.20,True,0.62,False,0.09,True,0.11
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,False,4.94,False,0.46,True,1.91,False,0.01,True,0.22
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,4.11,False,0.46,True,2.09,False,0.10,True,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,5.51,False,0.58,False,2.82,False,0.19,True,0.11
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,False,1.67,False,0.25,True,1.75,False,0.08,True,0.10
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,0.72,False,1.99,False,1.21,False,0.04,True,0.10
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,7.44,False,1.24,False,2.79,False,0.18,True,0.39


In [36]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['bike_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_bus'] < 0.5, 'bike_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_dep,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,2.77,False,0.81,False,2.65,False,0.23,True,0.16,True
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,4.55,False,0.22,True,3.23,False,0.09,True,0.15,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,1.72,True,0.20,True,0.62,False,0.09,True,0.11,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,4.94,False,0.46,True,1.91,False,0.01,True,0.22,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,4.11,False,0.46,True,2.09,False,0.10,True,0.11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,5.51,False,0.58,False,2.82,False,0.19,True,0.11,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,1.67,False,0.25,True,1.75,False,0.08,True,0.10,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.72,False,1.99,False,1.21,False,0.04,True,0.10,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,7.44,False,1.24,False,2.79,False,0.18,True,0.39,True


In [37]:
#確認
unique_values = df_training['bike_distance_less_than_500m'].unique()
print(unique_values)

[ True False]


mrt

In [38]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_mrt[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [39]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_bus 是包含公車站經緯度的 DataFrame
bus_coords = df_mrt[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371


In [40]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between mrt and house...')
ori = (0, 0)
bus_stn = df_mrt[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_mrt'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between mrt and house...


In [41]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_bike 欄位轉換成浮點數格式
df_training['d_mrt'] = df_training['d_mrt'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,dep_distance_less_than_500m,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,0.81,False,2.65,False,0.23,True,0.16,True,0.55
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,False,0.22,True,3.23,False,0.09,True,0.15,True,0.38
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,0.20,True,0.62,False,0.09,True,0.11,True,0.56
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,False,0.46,True,1.91,False,0.01,True,0.22,True,0.64
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,0.46,True,2.09,False,0.10,True,0.11,True,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,0.58,False,2.82,False,0.19,True,0.11,True,0.90
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,False,0.25,True,1.75,False,0.08,True,0.10,True,0.29
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,1.99,False,1.21,False,0.04,True,0.10,True,0.78
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,1.24,False,2.79,False,0.18,True,0.39,True,1.87


In [42]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['mrt_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_mrt'] < 0.5, 'mrt_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_tmarket,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.81,False,2.65,False,0.23,True,0.16,True,0.55,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.22,True,3.23,False,0.09,True,0.15,True,0.38,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.20,True,0.62,False,0.09,True,0.11,True,0.56,False
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.46,True,1.91,False,0.01,True,0.22,True,0.64,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.46,True,2.09,False,0.10,True,0.11,True,1.09,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.58,False,2.82,False,0.19,True,0.11,True,0.90,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.25,True,1.75,False,0.08,True,0.10,True,0.29,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,1.99,False,1.21,False,0.04,True,0.10,True,0.78,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,1.24,False,2.79,False,0.18,True,0.39,True,1.87,False


In [43]:
#確認
unique_values = df_training['mrt_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


train

In [44]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_train[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [45]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between train and house...')
ori = (0, 0)
bus_stn = df_train[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_train'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between train and house...


In [46]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_bike 欄位轉換成浮點數格式
df_training['d_train'] = df_training['d_train'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,tmarket_distance_less_than_500m,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,2.65,False,0.23,True,0.16,True,0.55,False,5.25
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,True,3.23,False,0.09,True,0.15,True,0.38,True,10.33
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,0.62,False,0.09,True,0.11,True,0.56,False,2.18
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,True,1.91,False,0.01,True,0.22,True,0.64,False,5.72
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,True,2.09,False,0.10,True,0.11,True,1.09,False,2.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,2.82,False,0.19,True,0.11,True,0.90,False,1.56
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,1.75,False,0.08,True,0.10,True,0.29,True,0.43
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,1.21,False,0.04,True,0.10,True,0.78,False,10.46
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,2.79,False,0.18,True,0.39,True,1.87,False,1.88


In [47]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['train_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_train'] < 0.5, 'train_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_bad,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,2.65,False,0.23,True,0.16,True,0.55,False,5.25,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,3.23,False,0.09,True,0.15,True,0.38,True,10.33,False
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.62,False,0.09,True,0.11,True,0.56,False,2.18,False
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,1.91,False,0.01,True,0.22,True,0.64,False,5.72,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,2.09,False,0.10,True,0.11,True,1.09,False,2.80,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,2.82,False,0.19,True,0.11,True,0.90,False,1.56,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,1.75,False,0.08,True,0.10,True,0.29,True,0.43,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,1.21,False,0.04,True,0.10,True,0.78,False,10.46,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,2.79,False,0.18,True,0.39,True,1.87,False,1.88,False


In [48]:
#確認
unique_values = df_training['train_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


交通便利性

In [49]:
import pandas as pd

# 假設 df 是你的 DataFrame
# 假設你想要判斷的欄位是 'column1', 'column2', 'column3'

# 使用 sum() 方法計算每一列中 true 的個數
df_training['true_count'] = df_training[['bike_distance_less_than_500m', 'bus_distance_less_than_500m', 'mrt_distance_less_than_500m','train_distance_less_than_500m']].sum(axis=1)

# 根據 true_count 判斷是否大於等於3，如果是則設為 3，等於2則設為2，小於2則設為1
df_training['convenient'] = df_training['true_count'].apply(lambda x: 3 if x >= 3 else (2 if x == 2 else 1))

# 移除中間產生的 'true_count' 欄位
df_training = df_training.drop(columns=['true_count'])
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,bad_distance_less_than_500m,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,0.23,True,0.16,True,0.55,False,5.25,False,2
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,False,0.09,True,0.15,True,0.38,True,10.33,False,3
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,False,0.09,True,0.11,True,0.56,False,2.18,False,2
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,False,0.01,True,0.22,True,0.64,False,5.72,False,2
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,0.10,True,0.11,True,1.09,False,2.80,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,0.19,True,0.11,True,0.90,False,1.56,False,2
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,False,0.08,True,0.10,True,0.29,True,0.43,True,3
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,0.04,True,0.10,True,0.78,False,10.46,False,2
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,0.18,True,0.39,True,1.87,False,1.88,False,2


In [50]:
df_post=pd.read_csv('./外部資料/郵局據點資料.csv')
df_ho=pd.read_csv('./外部資料/醫療機構基本資料.csv')
df_fin=pd.read_csv('./外部資料/金融機構基本資料.csv')

fin

In [51]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_fin[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [52]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between fin and house...')
ori = (0, 0)
bus_stn = df_fin[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_fin'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between fin and house...


In [53]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_ho 欄位轉換成浮點數格式
df_training['d_fin'] = df_training['d_fin'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_bus,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.23,True,0.16,True,0.55,False,5.25,False,2,0.89
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.09,True,0.15,True,0.38,True,10.33,False,3,0.13
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.09,True,0.11,True,0.56,False,2.18,False,2,0.15
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.01,True,0.22,True,0.64,False,5.72,False,2,0.14
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.10,True,0.11,True,1.09,False,2.80,False,2,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.19,True,0.11,True,0.90,False,1.56,False,2,0.37
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.08,True,0.10,True,0.29,True,0.43,True,3,0.11
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.04,True,0.10,True,0.78,False,10.46,False,2,0.21
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,0.18,True,0.39,True,1.87,False,1.88,False,2,0.66


In [54]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['fin_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_fin'] < 0.5, 'fin_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,bus_distance_less_than_500m,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,True,0.16,True,0.55,False,5.25,False,2,0.89,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,True,0.15,True,0.38,True,10.33,False,3,0.13,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,0.11,True,0.56,False,2.18,False,2,0.15,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,True,0.22,True,0.64,False,5.72,False,2,0.14,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,True,0.11,True,1.09,False,2.80,False,2,0.50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,True,0.11,True,0.90,False,1.56,False,2,0.37,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,0.10,True,0.29,True,0.43,True,3,0.11,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,True,0.10,True,0.78,False,10.46,False,2,0.21,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,True,0.39,True,1.87,False,1.88,False,2,0.66,False


In [55]:
#確認
unique_values = df_training['fin_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


post

In [56]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_post[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [57]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between post and house...')
ori = (0, 0)
bus_stn = df_post[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_post'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between post and house...


In [58]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_ho 欄位轉換成浮點數格式
df_training['d_post'] = df_training['d_post'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_bike,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m,d_post
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.16,True,0.55,False,5.25,False,2,0.89,False,0.99
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.15,True,0.38,True,10.33,False,3,0.13,True,0.37
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.11,True,0.56,False,2.18,False,2,0.15,True,0.44
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.22,True,0.64,False,5.72,False,2,0.14,True,0.61
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.11,True,1.09,False,2.80,False,2,0.50,False,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.11,True,0.90,False,1.56,False,2,0.37,True,0.49
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.10,True,0.29,True,0.43,True,3,0.11,True,0.28
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.10,True,0.78,False,10.46,False,2,0.21,True,0.47
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,0.39,True,1.87,False,1.88,False,2,0.66,False,1.12


In [59]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['post_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_post'] < 0.5, 'post_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,bike_distance_less_than_500m,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,True,0.55,False,5.25,False,2,0.89,False,0.99,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,True,0.38,True,10.33,False,3,0.13,True,0.37,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,0.56,False,2.18,False,2,0.15,True,0.44,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,True,0.64,False,5.72,False,2,0.14,True,0.61,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,True,1.09,False,2.80,False,2,0.50,False,0.55,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,True,0.90,False,1.56,False,2,0.37,True,0.49,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,0.29,True,0.43,True,3,0.11,True,0.28,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,True,0.78,False,10.46,False,2,0.21,True,0.47,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,True,1.87,False,1.88,False,2,0.66,False,1.12,False


In [60]:
#確認
unique_values = df_training['post_distance_less_than_500m'].unique()
print(unique_values)

[False  True]


hospital

In [61]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_ho[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [62]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between ho and house...')
ori = (0, 0)
bus_stn = df_ho[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_ho'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between ho and house...


In [63]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_ho 欄位轉換成浮點數格式
df_training['d_ho'] = df_training['d_ho'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_mrt,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.55,False,5.25,False,2,0.89,False,0.99,False,0.06
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.38,True,10.33,False,3,0.13,True,0.37,True,0.08
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.56,False,2.18,False,2,0.15,True,0.44,True,0.08
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.64,False,5.72,False,2,0.14,True,0.61,False,0.08
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,1.09,False,2.80,False,2,0.50,False,0.55,False,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.90,False,1.56,False,2,0.37,True,0.49,True,0.24
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.29,True,0.43,True,3,0.11,True,0.28,True,0.11
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.78,False,10.46,False,2,0.21,True,0.47,True,0.07
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,1.87,False,1.88,False,2,0.66,False,1.12,False,0.26


In [64]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['ho_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_ho'] < 0.5, 'ho_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,mrt_distance_less_than_500m,d_train,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho,ho_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,5.25,False,2,0.89,False,0.99,False,0.06,True
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,True,10.33,False,3,0.13,True,0.37,True,0.08,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,False,2.18,False,2,0.15,True,0.44,True,0.08,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,False,5.72,False,2,0.14,True,0.61,False,0.08,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,2.80,False,2,0.50,False,0.55,False,0.06,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,1.56,False,2,0.37,True,0.49,True,0.24,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,0.43,True,3,0.11,True,0.28,True,0.11,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,10.46,False,2,0.21,True,0.47,True,0.07,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,1.88,False,2,0.66,False,1.12,False,0.26,True


In [65]:
#確認
unique_values = df_training['ho_distance_less_than_500m'].unique()
print(unique_values)

[ True False]


便利商店

In [66]:
df_store=pd.read_csv('./外部資料/便利商店.csv')

In [67]:
missing_values = df_store.isna().sum()

print(missing_values)
# 假設 df_store 是你的 DataFrame
# 假設你想要刪除 'column_name' 中包含空值的行
df_store = df_store.dropna(subset=['lat'])
df_store = df_store.dropna(subset=['lng'])

公司名稱      0
分公司名稱     0
分公司地址     0
lat      15
lng      15
dtype: int64


In [68]:
# 假設 df_store 是包含便利商店經緯度的 DataFrame
store_coords = df_store[['lat', 'lng']].values
ball_tree = BallTree(np.radians(store_coords), metric='haversine')

def near_store(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return float(nearest_dist[0] * 6371 )  # 將距離轉換成米

In [69]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_store[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [70]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_store(row.lat, row.lon)

print('Calculating distance between store and house...')
ori = (0, 0)
bus_stn = df_store[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_store'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between store and house...


/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(nearest_dist[0] * 6371 )  # 將距離轉換成米
/var/folders/2p/bfbv31p50qg35jbmb1fczxq80000gn/T/ipykernel_57681/3588845307.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated Nu

In [71]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['store_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_store'] < 0.5, 'store_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,train_distance_less_than_500m,convenient,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho,ho_distance_less_than_500m,d_store,store_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,2,0.89,False,0.99,False,0.06,True,0.17,True
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,False,3,0.13,True,0.37,True,0.08,True,0.08,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,False,2,0.15,True,0.44,True,0.08,True,0.04,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,False,2,0.14,True,0.61,False,0.08,True,0.06,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,2,0.50,False,0.55,False,0.06,True,0.12,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,False,2,0.37,True,0.49,True,0.24,True,0.34,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,3,0.11,True,0.28,True,0.11,True,0.06,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,False,2,0.21,True,0.47,True,0.07,True,0.04,True
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,2,0.66,False,1.12,False,0.26,True,0.28,True


In [72]:
#確認
unique_values = df_training['store_distance_less_than_500m'].unique()
print(unique_values)

[ True False]


night market

In [73]:
df_nightmarket = pd.read_csv('./外部資料/夜市(永豐).csv')
df_nightmarket = df_nightmarket.rename(columns = {'Latitude':'lat','Longitude':'lng'})

In [74]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_nightmarket[['lat', 'lng']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [75]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between bus and house...')
ori = (0, 0)
bus_stn = df_nightmarket[['lat', 'lng']].rename(columns={'lat': 'lat', 'lng': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_nightmarket'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between bus and house...


In [76]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_nightmarket 欄位轉換成浮點數格式
df_training['d_nightmarket'] = df_training['d_nightmarket'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,convenient,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho,ho_distance_less_than_500m,d_store,store_distance_less_than_500m,d_nightmarket
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,2,0.89,False,0.99,False,0.06,True,0.17,True,0.74
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,3,0.13,True,0.37,True,0.08,True,0.08,True,3.20
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,2,0.15,True,0.44,True,0.08,True,0.04,True,0.05
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,2,0.14,True,0.61,False,0.08,True,0.06,True,0.88
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,2,0.50,False,0.55,False,0.06,True,0.12,True,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,2,0.37,True,0.49,True,0.24,True,0.34,True,0.59
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,3,0.11,True,0.28,True,0.11,True,0.06,True,0.78
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,2,0.21,True,0.47,True,0.07,True,0.04,True,2.31
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,2,0.66,False,1.12,False,0.26,True,0.28,True,2.61


In [77]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['nightmarket_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_nightmarket'] < 0.5, 'nightmarket_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_fin,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho,ho_distance_less_than_500m,d_store,store_distance_less_than_500m,d_nightmarket,nightmarket_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.89,False,0.99,False,0.06,True,0.17,True,0.74,False
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.13,True,0.37,True,0.08,True,0.08,True,3.20,False
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.15,True,0.44,True,0.08,True,0.04,True,0.05,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.14,True,0.61,False,0.08,True,0.06,True,0.88,False
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.50,False,0.55,False,0.06,True,0.12,True,1.18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.37,True,0.49,True,0.24,True,0.34,True,0.59,False
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.11,True,0.28,True,0.11,True,0.06,True,0.78,False
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.21,True,0.47,True,0.07,True,0.04,True,2.31,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,0.66,False,1.12,False,0.26,True,0.28,True,2.61,False


In [78]:
#確認
unique_values = df_training['nightmarket_distance_less_than_500m'].unique()
print(unique_values)


[False  True]


school

In [79]:
df_school = pd.read_csv('./private_clean_scl.csv')
df_school.columns

Index(['county', 'lon', 'lat', 'house_age', 'residence_housing',
       'congregate_housing', 'commercial_use', 'industrial_use', 'apartment',
       'building_low', 'building_high', 'RB', 'RC', 'SC', 'land_area',
       'building_area', 'main_building_area', 'balcony_area', 'auxiliary_area',
       'floor', 'total_floor', 'parking_area', 'parking_number', 'elementary',
       'total_elem', 'd_elementary', 'junior_high', 'total_junior', 'd_junior',
       'high_scl', 'total_hight', 'd_high', 'univr_scl', 'total_univr',
       'd_univr'],
      dtype='object')

In [80]:
df_school_filtered = df_school[['total_elem', 'd_elementary', 'total_junior', 'd_junior',
       'total_hight', 'd_high', 'total_univr','d_univr']]
df_school_filtered

,total_elem,d_elementary,total_junior,d_junior,total_hight,d_high,total_univr,d_univr
0,787,0.15,74,0.62,1485,0.39,7147,3.69
1,1544,0.40,1863,0.38,1165,0.38,3063,3.17
2,390,0.31,261,0.19,721,0.52,3980,0.55
3,2525,0.42,1147,0.61,1454,0.61,5254,6.32
4,3329,0.29,2178,0.60,1040,0.60,25760,2.04
...,...,...,...,...,...,...,...,...
5870,1221,0.17,1424,1.45,2106,1.85,9728,3.43
5871,726,0.48,301,0.66,292,0.56,6137,3.16
5872,1916,0.15,1210,0.32,1548,1.36,7517,3.25
5873,485,0.58,1424,1.87,2106,1.77,17568,2.68


In [81]:
df_temple=pd.read_csv('./外部資料/公廟.csv')

In [82]:
from sklearn.neighbors import BallTree
import numpy as np

# 假設 df_dep 是包含百貨公司經緯度的 DataFrame
bus_coords = df_temple[['Latitude', 'Longitude']].values
ball_tree = BallTree(np.radians(bus_coords), metric='haversine')

def near_Bus(x, y):
    global ori, ball_tree  # global vars
    ori = (x, y)           # 起點, i.e each row in df
    nearest_dist, _ = ball_tree.query([np.radians(ori)], k=1)
    return nearest_dist[0] * 6371   # 將距離轉換成米

In [83]:
from concurrent.futures import ThreadPoolExecutor

def parallel_distance_calculation(row):
    return near_Bus(row.lat, row.lon)

print('Calculating distance between bus and house...')
ori = (0, 0)
bus_stn = df_temple[['Latitude', 'Longitude']].rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}) # all bus station

with ThreadPoolExecutor() as executor:
    df_training['d_temple'] = list(executor.map(parallel_distance_calculation, df_training.itertuples(index=False)))


Calculating distance between bus and house...


In [84]:
# 使用科學記號來顯示
pd.options.display.float_format = '{:.2f}'.format

# 跑出你的計算
# ...

# 將 d_nightmarket 欄位轉換成浮點數格式
df_training['d_temple'] = df_training['d_temple'].astype(float)
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,fin_distance_less_than_500m,d_post,post_distance_less_than_500m,d_ho,ho_distance_less_than_500m,d_store,store_distance_less_than_500m,d_nightmarket,nightmarket_distance_less_than_500m,d_temple
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,False,0.99,False,0.06,True,0.17,True,0.74,False,0.18
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,True,0.37,True,0.08,True,0.08,True,3.20,False,0.14
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,True,0.44,True,0.08,True,0.04,True,0.05,True,0.07
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,True,0.61,False,0.08,True,0.06,True,0.88,False,0.32
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,False,0.55,False,0.06,True,0.12,True,1.18,False,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,True,0.49,True,0.24,True,0.34,True,0.59,False,0.26
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,True,0.28,True,0.11,True,0.06,True,0.78,False,0.15
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,True,0.47,True,0.07,True,0.04,True,2.31,False,0.70
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,False,1.12,False,0.26,True,0.28,True,2.61,False,1.44


In [87]:
# 假設df是你的資料框架，並且有一個名為'distance'的欄位表示距離
# 如果欄位名稱不同，請自行更換

# 新增一個欄位，預設為False
df_training['temple_distance_less_than_500m'] = False

# 將距離小於500公尺的設為True
df_training.loc[df_training['d_temple'] < 0.5, 'temple_distance_less_than_500m'] = True
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_temple,total_elem,d_elementary,total_junior,d_junior,total_hight,d_high,total_univr,d_univr,temple_distance_less_than_500m
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,0.18,787,0.15,74,0.62,1485,0.39,7147,3.69,True
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,0.14,1544,0.40,1863,0.38,1165,0.38,3063,3.17,True
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.07,390,0.31,261,0.19,721,0.52,3980,0.55,True
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,0.32,2525,0.42,1147,0.61,1454,0.61,5254,6.32,True
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,0.56,3329,0.29,2178,0.60,1040,0.60,25760,2.04,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,0.26,1221,0.17,1424,1.45,2106,1.85,9728,3.43,True
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,0.15,726,0.48,301,0.66,292,0.56,6137,3.16,True
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,0.70,1916,0.15,1210,0.32,1548,1.36,7517,3.25,False
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,1.44,485,0.58,1424,1.87,2106,1.77,17568,2.68,False


In [88]:
frames = [df_training,df_school_filtered]
df_training = pd.concat(frames,axis = 1)
df_training.columns

Index(['ID', 'lon', 'lat', 'house_age', 'residence_housing',
       'congregate_housing', 'commercial_use', 'industrial_use', 'apartment',
       'building_low', 'building_high', 'RB', 'RC', 'SC', 'land_area',
       'building_area', 'main_building_area', 'balcony_area', 'auxiliary_area',
       'floor', 'total_floor', 'parking_area', 'parking_number', 'd_park',
       'park_distance_less_than_500m', 'd_dep', 'dep_distance_less_than_500m',
       'd_tmarket', 'tmarket_distance_less_than_500m', 'd_bad',
       'bad_distance_less_than_500m', 'd_bus', 'bus_distance_less_than_500m',
       'd_bike', 'bike_distance_less_than_500m', 'd_mrt',
       'mrt_distance_less_than_500m', 'd_train',
       'train_distance_less_than_500m', 'convenient', 'd_fin',
       'fin_distance_less_than_500m', 'd_post', 'post_distance_less_than_500m',
       'd_ho', 'ho_distance_less_than_500m', 'd_store',
       'store_distance_less_than_500m', 'd_nightmarket',
       'nightmarket_distance_less_than_500m', 'd_te

In [89]:
df_training

,ID,lon,lat,house_age,residence_housing,congregate_housing,commercial_use,industrial_use,apartment,building_low,...,d_univr,temple_distance_less_than_500m,total_elem,d_elementary,total_junior,d_junior,total_hight,d_high,total_univr,d_univr
0,PR-1,120.32,22.59,27.92,1,0,0,0,0,0,...,3.69,True,787,0.15,74,0.62,1485,0.39,7147,3.69
1,PR-2,121.50,25.14,0.75,0,0,0,0,0,1,...,3.17,True,1544,0.40,1863,0.38,1165,0.38,3063,3.17
2,PR-3,121.52,25.07,43.92,1,0,0,0,0,1,...,0.55,True,390,0.31,261,0.19,721,0.52,3980,0.55
3,PR-4,121.48,25.09,45.42,1,0,0,0,1,0,...,6.32,True,2525,0.42,1147,0.61,1454,0.61,5254,6.32
4,PR-5,121.42,25.02,27.25,1,0,0,0,0,1,...,2.04,False,3329,0.29,2178,0.60,1040,0.60,25760,2.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,120.36,22.62,1.50,0,1,0,0,0,0,...,3.43,True,1221,0.17,1424,1.45,2106,1.85,9728,3.43
5871,PR-5872,121.58,25.05,17.58,0,0,1,0,0,0,...,3.16,True,726,0.48,301,0.66,292,0.56,6137,3.16
5872,PR-5873,121.37,25.07,27.08,1,0,0,0,0,0,...,3.25,False,1916,0.15,1210,0.32,1548,1.36,7517,3.25
5873,PR-5874,120.37,22.64,4.17,0,1,0,0,0,0,...,2.68,False,485,0.58,1424,1.87,2106,1.77,17568,2.68


In [90]:
df_private = df_training 

In [91]:
df_private.to_csv('private_with_alldata.csv')